In [1]:
import scanpy as sc
import sctk

sc.settings.set_figure_params(dpi=80)

In [ ]:
# this cell is tagged 'parameters' to use papermill
input_file = '/data/cephfs-1/work/projects/crc-patients-treatment-and-relapse/work/scRNA/raw_data/CE_SC_C_1/count/sample_filtered_feature_bc_matrix.h5'
output_dir = '/data/cephfs-1/work/groups/cubi/users/cemo10_c/scRNA/theislab_all_samples/preprocess_all_samples/CE_SC_C_1/'

In [ ]:
adata = sc.read_10x_h5(input_file)
adata

In [ ]:
adata.var_names_make_unique()
adata

In [ ]:
sctk.calculate_qc(adata)
adata

In [ ]:
sctk.cellwise_qc(adata)
adata

In [ ]:
adata.obs['cell_passed_qc'].sum()

In [ ]:
adata.uns['scautoqc_ranges']

In [ ]:
sctk.default_metric_params_df

In [ ]:
#present as columns in obs of the object
metrics_list = ["log1p_n_counts", "log1p_n_genes", "percent_mito", "percent_ribo"]
sctk.generate_qc_clusters(adata, metrics = metrics_list)
adata

In [ ]:
sc.pl.embedding(adata, "X_umap_qc", color=["qc_cluster", "log1p_n_counts", "percent_mito", "percent_ribo"], color_map="OrRd")

In [ ]:
sctk.clusterwise_qc(adata)
adata

In [ ]:
sc.pl.embedding(adata, "X_umap_qc", color=["cell_passed_qc", "cluster_passed_qc"])

In [ ]:
sctk.multi_resolution_cluster_qc(adata, metrics = metrics_list)
sc.pl.embedding(adata, "X_umap_qc", color=["cell_passed_qc",
                                           "cluster_passed_qc",
                                           "consensus_fraction",
                                           "consensus_passed_qc"])

In [ ]:
import seaborn as sns
p1 = sns.displot(adata.obs["n_counts"], bins=100, kde=False)
# sc.pl.violin(adata, 'total_counts')
p2 = sc.pl.violin(adata, "percent_mito")
p3 = sc.pl.scatter(adata, "n_counts", "n_genes", color="percent_mito")

In [ ]:
import os
os.makedirs(output_dir, exist_ok=True)
adata.uns['scautoqc_ranges'] = adata.uns['scautoqc_ranges'].to_dict()
adata.write(output_dir + "scAutoQC.h5ad")

In [14]:
# filter out cells that did not pass qc
adata = adata[adata.obs['cell_passed_qc'], :]

In [ ]:
import seaborn as sns
p1 = sns.displot(adata.obs["n_counts"], bins=100, kde=False)
# sc.pl.violin(adata, 'total_counts')
p2 = sc.pl.violin(adata, "percent_mito")
p3 = sc.pl.scatter(adata, "n_counts", "n_genes", color="percent_mito")